In [2]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
import random

# Projet: Modèles linéaires:  Adaline et Regression Logistique

Nous allons nous intéresser à l'implémentation d'un algorithme de descente de gradient pour trouver le meilleur paramètre d'un module Adaline ou de regression logistique.

Pout cela, on implémentera un algorithme de descente de gradient stochastique que nous avons vu au TP précédent et dont le pseudo-code peut être résumé comme suit:

```input: Train, eta, m, MaxEp, modele
init : w
epoque=0
while epoque<=MaxEp
    choisir un exemple (x,y) de Train de façon aléatoire
    calculer h = w*x
    calculer Loss(h, y)
    w <- w - eta*"gradient de Loss(h, y) par rapport à w"
    epoque <- epoque+1
output: w
```
où "eta" est le pas de la descente de gradient (exemple: eta=0.01).

Si on veut imprimer l'erreur tous les "m" pas de gradient:
```input: Train, eta, m, MaxEp, modele
init : w
epoque=0
while epoque<=MaxEp
    err = 0
    for i in range(m):
        choisir un exemple (x,y) de Train de façon aléatoire
        calculer h = w*x
        err += Loss(h, y)
        w <- w - eta*"gradient de Loss(h, y) par rapport à w"
    epoque <- epoque+1
    print(err)
output: w
```

Pour un poids $w$, on définit $h_\mathbf{w}(\mathbf{x})=w_0x_0+w_1x_1+...w_dx_d$. Pour chacun des deux modèles, et pour un exemple $(\mathbf{x},y)$, la prédiction $\hat{y}(\mathbf{w}, \mathbf{x})$ et la fonction de coût  $\mathcal{L}(\mathbf{w}, \mathbf{x})$ sont: 
- Adaline: $\hat{y}(\mathbf{w}, \mathbf{x}) = h_\mathbf{w}(x)$ et $$\mathcal{L}(\mathbf{w})=(y-\hat{y}(\mathbf{w},\mathbf{x}))^2=(y-h_\mathbf{w}(\mathbf{x}))^2,$$
- Régression logistique: $\hat{y}(w, x) = 1/(1+e^{-h_{\mathbf{w}}(\mathbf{x})})$ et $$\mathcal{L}(\mathbf{w}, x) = - y \log \hat{y}(\mathbf{w},\mathbf{x}) - (1-y)\log(1-\hat{y}(\mathbf{w},\mathbf{x})) = \log(1+e^{h_{\mathbf{w}}(\mathbf{x})})-yh_\mathbf{w}(\mathbf{x}),$$



In [3]:
###############################################
# Codage des fontions de perte et de gradient #
###############################################

# Adaline #
def h(w,x): # Prediction
    """
    w : vecteur de poids
    x : vecteur d'entrée
    Retourne le produit scalaire de w et x
    """
    return w@x

def loss_adaline(p,y):
    """
    p : prédiction
    y : valeur réelle
    Retourne la perte quadratique entre p et y
    """
    return (y-p)**2

def adaline_grad(w,x,y):
    """
    w : vecteur de poids
    x : vecteur d'entrée
    y : valeur réelle
    Retourne le gradient de la perte quadratique entre h(w,x) et y
    """
    return 2*(h(w,x) - y)*x



# Regression logistique #
def prediction_regression_logistique(w,x):
    """
    w : vecteur de poids
    x : vecteur d'entrée
    Retourne la prédiction de la régression logistique
    """
    return 1/(1 + np.exp(-h(w,x)))

def loss_regression_logistique(p,y):
    """
    p : prédiction
    y : valeur réelle
    Retourne la perte de la régression logistique
    """
    p = np.clip(p, 1e-10, 1-1e-10) # Pour éviter les erreurs de calcul
    return -y*np.log(p) - (1-y)*np.log(1-p)

def regression_logistique_grad(w,x,y):
    """
    w : vecteur de poids
    x : vecteur d'entrée
    y : valeur réelle
    Retourne le gradient de la perte de la régression logistique
    """
    return (prediction_regression_logistique(w,x)-y)*x

# Gradient
def gradient(grad, w, x, y):
    """
    Retourne le gradient de la fonction de perte
    """
    return grad(w,x,y)

Nous avons vu les gradients de ces fonctions en TD.

## Partie 1: implémentation de l'algorithme et exemple du "ET logique"

<font color='red'><b>Question 1:</b> le "ET logique".</font> Créer une liste de 4 éléments où chaque élément est un couple de la forme `[x,y]`, avec `x=[1,x1,x2]` et `y = x1 and x2`. Il y a 4 éléments car `x1` et `x2` peuvent chacun prendre la valeur `0` ou `1` (chacun de ces 4 éléments est une liste dont le premier élément est les attributs de l'exemple et le deuxième élément est la classe de l'exemple).

In [4]:
################################
# Creation de liste ET logique #
################################

liste = [0] * 4
for x1 in range(2):
    for x2 in range(2):
        liste[x1*1+x2*2] = [[1,x1,x2], x1 and x2] 

x_ET_logique = np.array([ei[0] for ei in liste])
y_ET_logique = np.array([ei[1] for ei in liste])

x_ET_logique, y_ET_logique


(array([[1, 0, 0],
        [1, 1, 0],
        [1, 0, 1],
        [1, 1, 1]]),
 array([0, 0, 0, 1]))

<font color='red'><b>Question 2:</b></font> Coder un algorithme de descente de gradient stochastique pour les modèles Adaline et le modèle de régression logistique et le faire tourner sur le modèle de "ET logique". 


In [5]:
#############################
# Fonction Gradient Descent #
#############################

import random as rd
    
# Stochastic Gradient Descent #
def SGD(X, Y, Maxep, eta, p, loss_function, g, err_printing=True):
    """
    Descente de gradient stochastique
    
    Entrees:
    X : tableau des features
    Y : tableau des labels / target variable
    Maxep : nombre d'epoques
    eta : fonction de taux d'apprentissage (learning rate)
    p : fonction de prediction
    loss_function : fonction de perte
    g : fonction de gradient
    err_printing : Boolean affichage des erreurs (True par defaut)
    
    Sorties:
    w : vecteur des poids
    """
    w = np.random.rand(len(X[0])) * 0.01
    for epoque in range(Maxep):
        i = rd.randint(0, len(Y) - 1)
        pred = p(w, X[i])
        if err_printing:
            print(loss_function(pred, Y[i]))
        w = w - eta(epoque) * g(w, X[i], Y[i])
    return w

def SGD_with_error_printing(X, Y, Maxep, eta, m, p, loss_function, g, err_printing=True):
    """
    Pareil que SGD mais la mise à jour des poids est faite m fois par époque
    L'erreur correspond à la somme sur les m exemples traités durant l'époque
    """
    w = np.random.rand(len(X[0])) * 0.01
    for epoque in range(Maxep):
        err = 0
        for _ in range(m):
            i = rd.randint(0, len(Y) - 1)
            pred = p(w, X[i])
            err += loss_function(pred, Y[i])
            w = w - eta(epoque) * g(w, X[i], Y[i])
        if err_printing:
            print("epoque ", epoque, ": ", err)
    return w

# Learning rate #   
def eta_001(t):
    return 0.01

def eta_01(t):
    return 0.1
#################


# Training Adaline on "ET logique"
w_adaline = SGD(x_ET_logique, y_ET_logique, 100, eta_001, h, loss_adaline, adaline_grad, False)
print("Final weights for Adaline on 'ET logique':", w_adaline)

# Training Logistic Regression on "ET logique"
w_logistic = SGD_with_error_printing(x_ET_logique, y_ET_logique, 100, eta_01, 2, prediction_regression_logistique, loss_regression_logistique, regression_logistique_grad, False)
print("Final weights for Logistic Regression on 'ET logique':", w_logistic)


Final weights for Adaline on 'ET logique': [0.07423324 0.21920745 0.22135396]
Final weights for Logistic Regression on 'ET logique': [-1.71685939  0.92508174  1.10415593]


Calculer le taux d'erreur de votre algorithme sur cette base (où une erreur est comptabilisé si la prédiction est plus proche de la fausse classe que de la vraie classe). 

In [6]:
#############################
# Calculer le taux d'erreur #
#############################

def calculate_error_rate(w, X, Y, prediction_function, rate_printing=False):
    """
    Calculer le taux d'erreur
    
    Entrees:
    w : vecteur des poids
    X : tableau des features
    Y : tableau des labels / target variable
    prediction_function : fonction de prediction
    rate_printing : Boolean affichage le taux d'erreur (False par defaut)
    
    Sorties:
    errors : taux d'erreur
    """
    errors = 0
    for i in range(len(Y)):
        pred = prediction_function(w, X[i])
        if rate_printing:
            print(f"P: {pred}, Y: {Y[i]}")
        if (pred >= 0.5 and Y[i] == 0) or (pred < 0.5 and Y[i] == 1):
            errors += 1
    return errors / len(Y)


# Function to calculate error rate multiple times
def calculate_multiple_error_rates(X, Y, num_iterations, model, prediction_function, eta, Maxep):
    """
    Calculer le taux d'erreur plusieurs fois
    
    Objectif:
    - Entrainer le modèle plusieurs fois
    - Calculer le taux d'erreur à chaque itération
    - Verifier la stabilité et la convergence de l'algorithme
    """
    if model == 'adaline':
        print("Training Adaline...") 
    elif model == 'logistic':
        print("Training Logistic Regression...")

    for _ in range(num_iterations):
        if model == 'adaline':
            w = SGD(X, Y, Maxep, eta, h, loss_adaline, adaline_grad, False)
        elif model == 'logistic':
            w = SGD(X, Y, Maxep, eta, prediction_regression_logistique, loss_regression_logistique, regression_logistique_grad, False)
        error_rate = calculate_error_rate(w, X, Y, prediction_function, False)
        print("Error rate:", error_rate)


# Calculate error rates for Adaline and Logistic Regression
num_iterations = 25

#####################################
# Calculate error rates for Adaline #
# calculate_multiple_error_rates(x_ET_logique, y_ET_logique, num_iterations, 'adaline', h, eta_001, 100)
calculate_multiple_error_rates(x_ET_logique, y_ET_logique, num_iterations, 'adaline', h, eta_01, 100)

#################################################
# Calculate error rates for Logistic Regression #
# calculate_multiple_error_rates(x_ET_logique, y_ET_logique, num_iterations, 'logistic', prediction_regression_logistique, eta_001, 100)
# calculate_multiple_error_rates(x_ET_logique, y_ET_logique, num_iterations, 'logistic', prediction_regression_logistique, eta_01, 100)
# calculate_multiple_error_rates(x_ET_logique, y_ET_logique, num_iterations, 'logistic', prediction_regression_logistique, eta_01, 200)
calculate_multiple_error_rates(x_ET_logique, y_ET_logique, num_iterations, 'logistic', prediction_regression_logistique, eta_01, 300)

Training Adaline...
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.25
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Training Logistic Regression...
Error rate: 0.0
Error rate: 0.0
Error rate: 0.25
Error rate: 0.0
Error rate: 0.25
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.25
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0


## Partie 2: premiers tests avec une base de donnée réelle

<font color='red'><b>Question 3:</b></font> Nous allons maintenant nous intéresser au comportement de ces modèles sur la base SONAR de la collection UCI (http://archive.ics.uci.edu/ml/index.php). Cette base contient 208 exemples en dimension 60 séparés par `,` et la dernière élément correspond à la classe de l'exemple.

    1. Télécharger la collection avec la fonction read_table de la librairie pandas (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_table.html). Les options nécessaires sont `sep=','` et `header=None`  
    2. Créer une liste de listes correspondant à la collection; pour cela initialiser la première liste et en parcourant chaque ligne de la matrice de données; créer une liste associée en remplaçant le dernier élément par `0` ou `+1` et insérer la dans la première liste. 
    Indication: Utiliser la fonction `loc`. 
    3. Écrire une fonction qui génère deux listes de données `x_train` (75%) and `x_test` (25%) en la mélangeant aléatoirement au préalable (indication: on pourra utiliser les fonctions `shuffle` de la librairie `random` et `train_test_split` de la librairie `sklearn.model_selection`)
    

In [7]:
# Sonar dataset #
sonar_data_fram = pd.read_table('Data/Sonar/sonar.all-data', sep = ',', header = None)
sonar_data_fram

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,0.0187,0.0346,0.0168,0.0177,0.0393,0.1630,0.2028,0.1694,0.2328,0.2684,...,0.0116,0.0098,0.0199,0.0033,0.0101,0.0065,0.0115,0.0193,0.0157,M
204,0.0323,0.0101,0.0298,0.0564,0.0760,0.0958,0.0990,0.1018,0.1030,0.2154,...,0.0061,0.0093,0.0135,0.0063,0.0063,0.0034,0.0032,0.0062,0.0067,M
205,0.0522,0.0437,0.0180,0.0292,0.0351,0.1171,0.1257,0.1178,0.1258,0.2529,...,0.0160,0.0029,0.0051,0.0062,0.0089,0.0140,0.0138,0.0077,0.0031,M
206,0.0303,0.0353,0.0490,0.0608,0.0167,0.1354,0.1465,0.1123,0.1945,0.2354,...,0.0086,0.0046,0.0126,0.0036,0.0035,0.0034,0.0079,0.0036,0.0048,M


- Fonction qui produit les listes de features et la liste de label
- Fonction qui génère deux listes de données `x_train` (75%) and `x_test` (25%) en la mélangeant aléatoirement au préalable

In [30]:
def number_of_lines_columns(dataFrame):
    """
    Retourne le nombre de lignes et de colonnes du dataframe
    """
    return dataFrame.shape[0], dataFrame.shape[1]

def produce_collection(dataFrame, numL, numC, targetC, classType=None):
    """
    Entree:
    dataFrame : le dataframe
    numL : nombre de lignes
    numC : nombre de colonnes
    targetC : colonne label
    classType : liste des classes (2 classes dans ce cas)
    
    Sortie:
    x : tableau des features
    y : tableau des labels 
    """
    x = np.zeros((numL, numC - 1))
    y = np.zeros(numL)
    
    for i in range(numC - 1):
        if (i < targetC):
            x[:,i] = dataFrame[i]
        elif (i > targetC):
            x[:,i-1] = dataFrame[i]
        else:
            continue
    
    if classType == None:
        for i in range(numL):
            y[i] = dataFrame[targetC][i]
    else:
        for i in range(numL):
            if dataFrame[targetC][i] == classType[0]: y[i] = 1
            elif dataFrame[targetC][i] == classType[1]: y[i] = 0
            else: 
                print("Erreur: classType doit contenir 2 classes")
                exit(0)
            
    return x, y.astype(int)

def generate_train_test_random(x, y):
    """
    Entree:
    x : tableau des features
    y : tableau des labels
    
    Sortie:
    x_train : tableau des features pour l'entrainement
    x_test : tableau des features pour le test
    y_train : tableau des labels pour l'entrainement
    y_test : tableau des labels pour le test
    """ 
    data = list(zip(x, y))
    rd.shuffle(data)
    new_X = np.array([i for i, j in data])
    new_Y = np.array([j for i, j in data])
    x_train, x_test, y_train, y_test = train_test_split(new_X, new_Y, test_size=0.25)
    return x_train, x_test, y_train, y_test

<font color='red'><b>Question 4:</b></font> Appliquer ces modèles sur cette base (on pourra prendre $MaxEp\approx1000$ et le pas d'apprentissage $\eta\approx0.1$) et en choisissant les bases Train et Test de façon aléatoire; Reporter l'erreur moyenne de ces modèles obtenues sur les exemples de donnés de "test"?  Refaire l'opération 3 fois avec trois randomisations différentes. 


In [23]:
# Sonar Data Frame #
sonar_num_line, sonar_num_col = number_of_lines_columns(sonar_data_fram)
x_sonar, y_sonar = produce_collection(sonar_data_fram, sonar_num_line, sonar_num_col, sonar_num_col - 1, ['M', 'R'])


MaxEp = 1000
for _ in range(3):
    x_train, x_test, y_train, y_test = generate_train_test_random(x_sonar, y_sonar)
    out = "iter " + str(_) + ":\n"   
    out += "Training Adaline on sonar data...\n"
    w_adaline = SGD(x_train, y_train, MaxEp, eta_001, h, loss_adaline, adaline_grad, False)
    error_rate = []
    for x_i, y_i in zip(x_test, y_test):
        error_rate.append(loss_adaline(h(w_adaline, x_i), y_i))
    out += "Error rate: " + str(np.mean(error_rate)) + "\n"
    
    out += "Training Logistic Regression on sonar data...\n"
    w_logistic = SGD(x_train, y_train, MaxEp, eta_01, prediction_regression_logistique, loss_regression_logistique, regression_logistique_grad, False)
    error_rate = []
    for x_i, y_i in zip(x_test, y_test):
        error_rate.append(loss_regression_logistique(prediction_regression_logistique(w_logistic, x_i), y_i))
    out += "Error rate: " + str(np.mean(error_rate)) + "\n"
    print(out)

iter 0:
Training Adaline on sonar data...
Error rate: 0.1403899412782531
Training Logistic Regression on sonar data...
Error rate: 0.5813312470639169

iter 1:
Training Adaline on sonar data...
Error rate: 0.18468670537593143
Training Logistic Regression on sonar data...
Error rate: 0.5657948500648395

iter 2:
Training Adaline on sonar data...
Error rate: 0.16183390377785084
Training Logistic Regression on sonar data...
Error rate: 0.4482578919615845



  | Collection | Adaline     | Régression Logistique |
  |------------|-------------|-----------------------|
  |   SONAR (réplica 1)   |      $\approx19$%       |  $\approx50$%         |
  |   SONAR (réplica 2)   |       $\approx16.8$%    | $\approx50$%        |
  |   SONAR (réplica 3)   |       $\approx18.8$%    |       $\approx60$%    |


## Partie 3: normalisation

Nous allons étudier l'impact de la nomralisation sur les prédictions. Pour cela nous considérons deux stratégies de normalisation communément utilisées dans la littérature:
* Stratégie <i>max</i>: consiste à normaliser chaque caractéristique du vecteur réprésentatif d'une observation par la valeur maximale de cette caractéristiques
* Stratégie <i>norme</i>: consiste à normaliser chaque caractéristique du vecteur réprésentatif d'une observation par la norme de ce vecteur.

Nous considérons ces trois autres collections de la base UCI:

        * https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29
        * https://archive.ics.uci.edu/ml/datasets/spambase
        * https://archive.ics.uci.edu/ml/datasets/ionosphere

        

In [20]:
# Importer dataset #
breast_cancer_df = pd.read_table('Data/BreastCancer/wdbc.data', header=None, sep=',')
ionosphere_df = pd.read_csv('Data/Ionosphere/ionosphere.data', header=None, sep=',')
spambase_df = pd.read_csv('Data/Spambase/spambase.data', header=None, sep=',')

breast_cancer_df
ionosphere_df
spambase_df

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.000,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.010,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4596,0.31,0.00,0.62,0.0,0.00,0.31,0.00,0.00,0.00,0.00,...,0.000,0.232,0.0,0.000,0.000,0.000,1.142,3,88,0
4597,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.353,0.000,0.000,1.555,4,14,0
4598,0.30,0.00,0.30,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.102,0.718,0.0,0.000,0.000,0.000,1.404,6,118,0
4599,0.96,0.00,0.00,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.057,0.0,0.000,0.000,0.000,1.147,5,78,0



<font color='red'><b>Question 5:</b></font> Ecrire une fonction qui prend en entrée la collection des données et qui retourne la collections normalisée suivant les stratégies <i>max</i> et <i>norme</i>. 

In [ ]:
def normalizer_collection(collection, strategies = 'max'):
    collection_clone = collection.select_dtypes(include=[np.number]).astype(float).copy()
    
    if strategies == 'max':
        max_value = np.max(np.abs(collection_clone), axis=0)
        max_value[max_value == 0] = 1
        collection_clone = collection_clone / max_value
    
    # A COMPLETER #
    # X = - ecarttype/moyen
    elif strategies == 'norme':
        normes = np.linalg.norm(collection_clone, axis=0)
        normes[normes == 0] = 1
        collection_clone = collection_clone / normes
    else:
        raise ValueError("Invalid normalization strategy")
    # A COMPLETER #
    collection[collection_clone.columns] = collection_clone
    return collection

<font color='red'><b>Question 6:</b></font> Compléter les tableaux comparatifs suivants en repertant les erreurs moyennes sur 20 lancements des modèles de l'Adaline et de la Régression Logistique et pour les trois cas:

 '*' Les vecteurs ne sont pas normalisés
     
  | Collection |   Adaline   |  Régression Logistique |
  |------------|-------------|------------------------|
  |   BREAST   |             |                        |
  |   IONO     |             |                        |
  |   SONAR    |             |                        |
  |   SPAM     |             |                        |




In [ ]:
print("collection non normalized")
x_spambase, y_spambase = produce_collection(spambase_df, spambase_df.shape[0], spambase_df.shape[1], spambase_df.shape[1] - 1)




collection non normalized


array([1, 1, 1, ..., 0, 0, 0])

In [16]:
print("collection normalized with norme")


collection normalized with norme


 
 $^n$ Normalisation suivant la stratégie <i>norme</i>
     
  | Collection |   Adaline   |  Régression Logistique |
  |------------|-------------|------------------------|
  |   BREAST   |             |                        |
  |   IONO     |             |                        |
  |   SONAR    |             |                        |
  |   SPAM     |             |                        |

  


In [17]:
print("collection normalized with max")

collection normalized with max


 $^m$ Normalisation suivant la stratégie <i>max</i>
    
  | Collection |   Adaline   |  Régression Logistique |
  |------------|-------------|------------------------|
  |   BREAST   |             |                        |
  |   IONO     |             |                        |
  |   SONAR    |             |                        |
  |   SPAM     |             |                        |
